In [12]:
import json
import pickle
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.preprocessing import LabelEncoder
from pprint import pprint
from utils import *

tagger = MeCab.Tagger()

In [13]:
input_data_path = '../data/input_data/'
output_data_path = '../data/output_data/'
outer_data_path = '../data/outer_data/'

In [14]:
with open(input_data_path+'train_aozora_data.json') as f:
    novels_json = json.load(f)

novel_id_l = []
author_l = []
for author_data in novels_json:
    for novel in author_data['novels']:
        novel_id_l.append(novel['content_id'])
        author_l.append(author_data['author_id'])

In [4]:
model_path = '../data/outer_data/chive-1.2-mc5.kv'
w2v_model = KeyedVectors.load(model_path)

In [15]:
with open(input_data_path+'aozora_contents.json', encoding='utf-8') as f:
    novel_text_dic = json.load(f)

test_l = []
i = 0
for novel_content_id, author_id in zip(novel_id_l, author_l):
    novel_content = novel_text_dic[novel_content_id].replace('\n', '').replace('\r', '').replace('\u3000', '')
    novel_word_l = doc_wakati(novel_content)
    text_vec = text_to_vec(novel_word_l, w2v_model)

    if text_vec is None:
        continue

    test_l.append([author_id] + list(text_vec))
    print('\r'+str(i), end='')
    i+=1

train_df = pd.DataFrame(test_l, columns=['author_id'] + [i for i in range(len(text_vec))])

13861

In [16]:
with open(input_data_path+'aozora_authors.json', encoding='utf-8') as f:
    authors_dict = json.load(f)
le = LabelEncoder()
le.fit(list(authors_dict.keys()))
train_df['author_id'] = le.transform(train_df['author_id'])

In [20]:
with open(output_data_path+'train_df.pickle', 'wb') as f:
    pickle.dump(train_df, f)

In [23]:
train_df['author_id'].max()

869

In [24]:
#ここからテストデータの前処理
with open(input_data_path+'test_q.json', encoding='utf-8') as f:
    test_json = json.load(f)
with open(input_data_path+'test_aozora_data.json', encoding='utf-8')as f:
    title2con_dict = json.load(f)

candis_list = []
test_l = []
for test_data in test_json:
    candis_list.append(le.transform(test_data['candidates']))
    novel_content_id = title2con_dict[test_data['title_id']]
    novel_content = novel_text_dic[novel_content_id].replace('\n', '').replace('\r', '').replace('\u3000', '')
    novel_word_l = doc_wakati(novel_content)
    text_vec = text_to_vec(novel_word_l, w2v_model)

    if text_vec is None:
        continue

    test_l.append(list(text_vec))

test_df = pd.DataFrame(test_l, columns=[i for i in range(len(text_vec))])

In [30]:
with open(output_data_path+'test_df.pickle', 'wb') as f:
    pickle.dump(test_df, f)

with open(output_data_path+'candis_list.pickle', 'wb') as f:
    pickle.dump(list(map(list, candis_list)), f)